<a href="https://colab.research.google.com/github/Aku02/Awesome-pytorch-list/blob/master/GLR2020/Google_landmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
pic=np.load('/content/gdrive/My Drive/google_landmark/pictures.npy')
df=pd.read_csv('/content/gdrive/My Drive/google_landmark/output.csv',index_col=[0])
idx=[]
a=df['labels'].value_counts()
for i,j in zip(a.index,a.values):
  smp=int(0.5*j)
  if int(smp)>5:
    smp=5
  idx.extend(df.loc[df['labels']==i]['id'].sample(smp))
df['val']=df['id'].isin(idx).astype(int)
val=df.loc[df['val']==1]
trn=df.loc[df['val']==0]
val_pictures=pic[val.index]
trn_pictures=pic[trn.index]
trn=trn.reset_index(drop=True)
val=val.reset_index(drop=True)

In [ ]:
pip install pydegensac

In [ ]:
trn['i']=1
trn['counts']=trn.groupby(['labels'])['i'].transform(pd.Series.sum)
trn=trn.loc[trn['counts']>3]
trn_pictures=trn_pictures[trn.index]

In [ ]:

def load_labelmap():
  dk={}
  for _,i in trn.iterrows():
    dk[i['id']]=i['labels']
  return dk
labelmap=load_labelmap()

import copy
import csv
import gc
import operator
import os
import pathlib
import shutil

import numpy as np
import PIL
import pydegensac
from scipy import spatial
import tensorflow as tf

# Dataset parameters:
INPUT_DIR = os.path.join('..', 'input')

DATASET_DIR = os.path.join(INPUT_DIR, 'landmark-recognition-2020')
TEST_IMAGE_DIR = os.path.join(DATASET_DIR, 'test')
TRAIN_IMAGE_DIR = os.path.join(DATASET_DIR, 'train')
TRAIN_LABELMAP_PATH = os.path.join(DATASET_DIR, 'train.csv')

# DEBUGGING PARAMS:
NUM_PUBLIC_TRAIN_IMAGES = 1580470 # Used to detect if in session or re-run.
MAX_NUM_EMBEDDINGS = -1  # Set to > 1 to subsample dataset while debugging.

# Retrieval & re-ranking parameters:
NUM_TO_RERANK = 15
TOP_K = 4 # Number of retrieved images used to make prediction for a test image.

# RANSAC parameters:
MAX_INLIER_SCORE = 10
MAX_REPROJECTION_ERROR = 4.0
MAX_RANSAC_ITERATIONS = 1000
HOMOGRAPHY_CONFIDENCE = 0.99

# DELG model:
SAVED_MODEL_DIR = '/content/gdrive/My Drive/google_landmark/local_and_global'
DELG_MODEL = tf.saved_model.load(SAVED_MODEL_DIR)
DELG_IMAGE_SCALES_TENSOR = tf.convert_to_tensor([0.70710677, 1.0, 1.4142135])
DELG_SCORE_THRESHOLD_TENSOR = tf.constant(175.)
DELG_INPUT_TENSOR_NAMES = [
    'input_image:0', 'input_scales:0', 'input_abs_thres:0'
]

# Global feature extraction:
NUM_EMBEDDING_DIMENSIONS = 2048
GLOBAL_FEATURE_EXTRACTION_FN = DELG_MODEL.prune(DELG_INPUT_TENSOR_NAMES,
                                                ['global_descriptors:0'])

# Local feature extraction:
LOCAL_FEATURE_NUM_TENSOR = tf.constant(1000)
LOCAL_FEATURE_EXTRACTION_FN = DELG_MODEL.prune(
    DELG_INPUT_TENSOR_NAMES + ['input_max_feature_num:0'],
    ['boxes:0', 'features:0'])



In [ ]:

def extract_global_features(type):
  if type=='train':
    df=trn
    pic=trn_pictures
  if type=='test':
    df=val
    pic=val_pictures

  num_embeddings = df.shape[0]
  if MAX_NUM_EMBEDDINGS > 0:
    num_embeddings = min(MAX_NUM_EMBEDDINGS, num_embeddings)

  embeddings = np.empty((num_embeddings, NUM_EMBEDDING_DIMENSIONS))

  for i in tqdm(range(df.shape[0])):
    if i >= num_embeddings:
      break

    # ids[i] = int(image_path.name.split('.')[0], 16)

    image_tensor =tf.convert_to_tensor(pic[i])

    features = GLOBAL_FEATURE_EXTRACTION_FN(image_tensor,
                                            DELG_IMAGE_SCALES_TENSOR,
                                            DELG_SCORE_THRESHOLD_TENSOR)
    embeddings[i, :] = tf.nn.l2_normalize(
        tf.reduce_sum(features[0], axis=0, name='sum_pooling'),
        axis=0,
        name='final_l2_normalization').numpy()

  return embeddings
from tqdm import tqdm
test_embeddings = extract_global_features('test')
test_ids=val['id'].values
train_embeddings = extract_global_features('train')
train_ids=trn['id'].values
train_ids_labels_and_scores = [None] * test_embeddings.shape[0]


  0%|          | 1/633 [00:00<09:30,  1.11it/s]

(3, 2048)



  0%|          | 2/633 [00:01<08:25,  1.25it/s]

(3, 2048)



  0%|          | 3/633 [00:02<07:37,  1.38it/s]

...

(3, 2048)



 99%|█████████▉| 198/199 [01:50<00:00,  1.83it/s]

(3, 2048)



100%|██████████| 199/199 [01:50<00:00,  1.79it/s]

(3, 2048)


In [ ]:
trn['labels'].unique()

array([138982, 126637,  20409,  83144, 113209, 177870, 194914, 149980,
       139894,   1924, 176528,   9070,  20120,  46705, 168098, 161902,
        36748, 176018, 120734,  10419,  45428,  14915,  41648, 165596,
        47378, 187779,  38482])

In [ ]:
dk={}
for lbl,data in trn.groupby(['labels']):
  data_pictures=trn_pictures[data.index]
  data_emb=train_embeddings[data.index]
  sh=data_emb.shape[0]
  tmp={}
  for en,index in enumerate(data.index):
    e1=data_emb[en]
    dist = spatial.distance.cdist(e1.reshape(1,2048),data_emb,'cosine')[0]
    idxs=np.argsort(dist)[:100]
    dist=dist[idxs]
    distances=[]
    for dist,idx in zip(dist,idxs):
      if dist<0.5:
        distances.append(idx)
    num=len(distances)
    ls=np.zeros(101)
    ls[0]=num
    ls[1:num+1]=np.asarray(data.index)[distances]
    tmp[str(index)]=ls
  dk[str(lbl)]=tmp
  


  

In [ ]:
pd.DataFrame(dk['83144']).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
86,2.0,86.0,89.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
87,2.0,87.0,91.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
88,1.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
89,2.0,89.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
90,1.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
91,2.0,91.0,87.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92,2.0,92.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
93,2.0,93.0,95.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,2.0,94.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,2.0,95.0,93.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dk={}
for lbl in trn['labels'].unique():
  dk[str(lbl)]={}
  lbl,data=lbl,trn.loc[trn['labels']==lbl]
  for i,e1 in enumerate(data_emb):
    count=-1
    ls=[]
    for j,e2 in enumerate(data_emb):
      distances = spatial.distance.cdist(e1.reshape(1,2048),e2.reshape(1,2048),'cosine')[0]
      if distances>0.5:
        count+=1
        ls[count+1]=j
    ls[0]=count

  tot.append(ls)

In [ ]:
for test_index in tqdm(range(test_embeddings.shape[0])):
    distances = spatial.distance.cdist(
        test_embeddings[np.newaxis, test_index, :], train_embeddings,
        'cosine')[0]
    partition = np.argpartition(distances, NUM_TO_RERANK)[:NUM_TO_RERANK]

    nearest = sorted([(train_ids[p], distances[p]) for p in partition],
                     key=lambda x: x[1])

    train_ids_labels_and_scores[test_index] = [
        (list(train_ids).index(train_id),train_id, labelmap[train_id], 1. - cosine_distance)
        for train_id, cosine_distance in nearest
    ]

del test_embeddings
del train_embeddings
del labelmap
gc.collect()


In [ ]:

def get_prediction_map(test_ids, train_ids_labels_and_scores):
  """Makes dict from test ids and ranked training ids, labels, scores."""

  prediction_map = dict()

  for test_index, test_id in enumerate(test_ids):

    aggregate_scores = {}
    for _,_, label, score in train_ids_labels_and_scores[test_index][:TOP_K]:
      if label not in aggregate_scores:
        aggregate_scores[label] = 0
      aggregate_scores[label] += score

    label, score = max(aggregate_scores.items(), key=operator.itemgetter(1))

    prediction_map[test_id] = {'score': score, 'class': label}

  return prediction_map
pre_verification_predictions = get_prediction_map(
      test_ids, train_ids_labels_and_scores)

def get_putative_matching_keypoints(test_keypoints,
                                    test_descriptors,
                                    train_keypoints,
                                    train_descriptors,
                                    max_distance=0.9):
  """Finds matches from `test_descriptors` to KD-tree of `train_descriptors`."""

  train_descriptor_tree = spatial.cKDTree(train_descriptors)
  _, matches = train_descriptor_tree.query(
      test_descriptors, distance_upper_bound=max_distance)

  test_kp_count = test_keypoints.shape[0]
  train_kp_count = train_keypoints.shape[0]

  test_matching_keypoints = np.array([
      test_keypoints[i,]
      for i in range(test_kp_count)
      if matches[i] != train_kp_count
  ])
  train_matching_keypoints = np.array([
      train_keypoints[matches[i],]
      for i in range(test_kp_count)
      if matches[i] != train_kp_count
  ])

  return test_matching_keypoints, train_matching_keypoints


def get_num_inliers(test_keypoints, test_descriptors, train_keypoints,
                    train_descriptors):
  """Returns the number of RANSAC inliers."""

  test_match_kp, train_match_kp = get_putative_matching_keypoints(
      test_keypoints, test_descriptors, train_keypoints, train_descriptors)

  if test_match_kp.shape[
      0] <= 4:  # Min keypoints supported by `pydegensac.findHomography()`
    return 0

  try:
    _, mask = pydegensac.findHomography(test_match_kp, train_match_kp,
                                        MAX_REPROJECTION_ERROR,
                                        HOMOGRAPHY_CONFIDENCE,
                                        MAX_RANSAC_ITERATIONS)
  except np.linalg.LinAlgError:  # When det(H)=0, can't invert matrix.
    return 0

  return int(copy.deepcopy(mask).astype(np.float32).sum())


def extract_local_features(image_tensor):
  """Extracts local features for the given `image_path`."""


  features = LOCAL_FEATURE_EXTRACTION_FN(image_tensor, DELG_IMAGE_SCALES_TENSOR,
                                         DELG_SCORE_THRESHOLD_TENSOR,
                                         LOCAL_FEATURE_NUM_TENSOR)

  # Shape: (N, 2)
  keypoints = tf.divide(
      tf.add(
          tf.gather(features[0], [0, 1], axis=1),
          tf.gather(features[0], [2, 3], axis=1)), 2.0).numpy()

  # Shape: (N, 128)
  descriptors = tf.nn.l2_normalize(
      features[1], axis=1, name='l2_normalization').numpy()

  return keypoints, descriptors
def rescore_and_rerank_by_num_inliers(test_image_id,
                                      train_ids_labels_and_scores):
  """Returns rescored and sorted training images by local feature extraction."""

  # test_image_path = get_image_path('test', test_image_id)
  val_pic=tf.convert_to_tensor(val_pictures[test_image_id])
  test_keypoints, test_descriptors = extract_local_features(val_pic)

  for i in range(len(train_ids_labels_and_scores)):
    index,train_image_id, label, global_score = train_ids_labels_and_scores[i]

    trn_pic = tf.convert_to_tensor(trn_pictures[index])
    train_keypoints, train_descriptors = extract_local_features(
        trn_pic)

    num_inliers = get_num_inliers(test_keypoints, test_descriptors,
                                      train_keypoints, train_descriptors)
    total_score = get_total_score(num_inliers, global_score)
    train_ids_labels_and_scores[i] = (index,train_image_id, label, total_score)

  train_ids_labels_and_scores.sort(key=lambda x: x[2], reverse=True)

  return train_ids_labels_and_scores

def get_total_score(num_inliers, global_score):
  local_score = min(num_inliers, MAX_INLIER_SCORE) / MAX_INLIER_SCORE
  return local_score + global_score
for test_index, test_id in tqdm(enumerate(test_ids)):
    train_ids_labels_and_scores[test_index] = rescore_and_rerank_by_num_inliers(
        test_index, train_ids_labels_and_scores[test_index])
post_verification_predictions = get_prediction_map(
      test_ids, train_ids_labels_and_scores)

In [ ]:
post_verification_predictions